In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jan_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jan_dict_JSON = json.load(json_file)

In [10]:
week_day_Jan_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]

In [11]:
link_day_minute_Jan_list = []
for link_idx in range(24):
    for day in week_day_Jan_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jan_list.append(link_day_minute_Jan_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Jan_list)

63360

In [13]:
x = np.matrix(link_day_minute_Jan_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [15]:
x[:,:2]

matrix([[  969907.9571843 ,   968063.94595178],
        [  978395.30237817,  1008735.41555166],
        [ 1853976.53538362,  1841098.31817489],
        [ 1947512.47327385,  1931992.27840584],
        [  481868.81225345,   473671.12220579],
        [  589026.19856293,   597773.88868234],
        [ 1136825.10779262,  1140282.48913682],
        [ 1158008.53508594,  1158527.98068668],
        [ 1061499.20245405,  1113849.97959017],
        [ 1143015.38321959,  1098606.53865478],
        [ 1185708.91032356,  1322823.55988809],
        [ 1040088.21471554,  1036032.51066563],
        [  267642.72358573,   267370.96272315],
        [  272883.84436931,   277479.63135155],
        [  429061.8335527 ,   458372.76314855],
        [  431050.070705  ,   403217.04251251],
        [  620537.01738166,   622074.11365878],
        [  624018.24026134,   622681.45771696],
        [  994217.10584384,   989083.19887004],
        [  899127.75384335,   913918.28240208],
        [ 1441120.05505741,  1475238.814

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [2e-01, 2e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -3.87785759e+07  0.00000000e+00  6.10e+03 2.41e+03  1.00e+06     0s
   1  -3.46192434e+07 -1.11467415e+07  2.04e+03 8.04e+02  3.70e+05     0s
   2  -2.77135497e+07 -3.23336243e+07  9.50e+01 3.75e+01  1.34e+05     0s
   3  -2.81458368e+07 -2.86900502e+07  9.50e-05 3.75e-05  9.7

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jan_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[8.654528630957624e-10,
 2.6941135274426153e-05,
 617127.8800686705,
 4.238886394489108e-07,
 2.5824156296823386e-05,
 771491.2454546142,
 3.450259797093346e-07,
 2.2158419994432367e-08,
 5.3590955497471624e-11,
 623582.3151787068,
 5.358523977120299e-11,
 5.359095966533033e-11,
 5.359174220837859e-11,
 5.394553479987468e-11,
 2036729.7505796643,
 7.610143467279479e-10,
 74820.4330423494,
 4.992962032622722e-07,
 475920.9461802088,
 49013.77843947752,
 4.1067605234453583e-07,
 409955.0594647531,
 835279.9462625613,
 3.554531923660731e-07,
 3.7631651378605777e-07,
 3.682379746510465e-07,
 127687.70885510807,
 439712.97926154634,
 1.0284055173019526e-06,
 7.604510605951055e-10,
 255237.94136810955,
 40058.34297669268,
 399371.2334814501,
 4.420152742978247e-08,
 233839.35160039744,
 3.71303653693973e-07,
 7.59457370739178e-10,
 319894.4943824869,
 2.538385756077426e-07,
 341134.28488374525,
 833375.7177087883,
 3.4444558628845667e-07,
 9.506561287662778e-08,
 7.549703362021795e-10,
 1445